In [246]:
import pandas as pd
import sqlite3

создаем подключение к базе данных с помощью библиотеки sqlite3

In [247]:
db = sqlite3.connect('../data/checking-logs.sqlite', check_same_thread=False)

получить схему таблицы test

In [248]:
pd.io.sql.read_sql('PRAGMA table_info(test)', db)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


получить только первые 10 строк таблицы test, чтобы проверить, как выглядит таблица

In [249]:
pd.io.sql.read_sql('SELECT * FROM test LIMIT 10', db)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [250]:
pd.io.sql.read_sql('SELECT julianday(first_commit_ts) FROM test LIMIT 10', db)

,julianday(first_commit_ts)
0,2.458966e+06
1,2.458966e+06
2,2.458972e+06
3,2.458987e+06
4,2.458990e+06
5,2.458984e+06
6,2.458965e+06
7,2.458965e+06
8,2.458971e+06
9,2.458989e+06


## найти среди всех пользователей минимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лабораторной работы, используя только один запрос

сделайте это, присоединившись к таблице deadlines
разница должна отображаться в часах
не принимайте во внимание лабораторную работу «проект 1», у нее более длительные сроки выполнения, и она будет выделяться
значение должно быть сохранено в фрейме данных df_min с соответствующим uid

In [251]:
pd.io.sql.read_sql('PRAGMA table_info(deadlines)', db)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [252]:
pd.io.sql.read_sql('SELECT * FROM deadlines LIMIT 10', db)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [253]:
query = """
SELECT 
    test.uid AS uid,
    MIN(cast(( julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24 as integer)) AS min_diff

FROM 
    test
LEFT JOIN 
    deadlines
ON 
    test.labname = deadlines.labs
WHERE 
    test.labname != 'project1'
GROUP BY 
    test.uid
ORDER BY min_diff ASC
LIMIT 1;

"""
df_min = pd.io.sql.read_sql(query, db)
df_min

,uid,min_diff
0,user_16,-206


## сделайте то же самое, но для максимума, используя только один запрос, имя dataframe будет df_max

In [254]:
query = """
SELECT 
    test.uid AS uid,
    MAX(cast((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24 as integer)) AS max_diff

FROM 
    test
LEFT JOIN 
    deadlines
ON 
    test.labname = deadlines.labs
WHERE 
    test.labname != 'project1'
GROUP BY 
    test.uid
ORDER BY max_diff ASC
LIMIT 1;
"""
df_max = pd.io.sql.read_sql(query, db)
df_max

,uid,max_diff
0,user_11,2


## сделайте то же самое, но для среднего значения, используя только один запрос, на этот раз ваш фрейм данных не должен включать столбец uid, а имя фрейма данных — df_avg

In [255]:
query = """
SELECT 
    test.uid AS uid,
    AVG(cast((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24 as integer)) AS avg_diff
FROM 
    test
LEFT JOIN 
    deadlines
ON 
    test.labname = deadlines.labs
WHERE 
    test.labname != 'project1'

"""
df_avg = pd.io.sql.read_sql(query, db)
df_avg

,uid,avg_diff
0,user_1,99.526316


## мы хотим проверить гипотезу о том, что пользователи, которые посетили новостную ленту всего несколько раз, имеют более низкую дельту между первым коммитом и дедлайном. Для этого нужно рассчитать коэффициент корреляции между количеством просмотров страниц и разницей

1. Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews
2. uid — это uid, которые существуют в тесте
3. avg_diff — это средняя разница между первым коммитом и крайним сроком выполнения лабораторных работ на пользователя.
4. просмотры страниц — это количество посещений новостной ленты на одного пользователя
5. не принимайте во внимание лабораторную работу «проект1»
6. сохраните его в dataframe views_diff
7. используйте метод Pandas corr() для расчета коэффициента корреляции между количеством просмотров страниц и разницей

In [261]:
views_diff_query = """
SELECT
    test.uid AS uid,
    AVG(cast((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24 as integer)) AS avg_diff,
    COUNT(test.uid) as pageviews
FROM
    test
INNER JOIN
    deadlines
ON
    test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
GROUP BY
    test.uid;
"""
views_diff = pd.io.sql.read_sql(views_diff_query, db)
views_diff


,uid,avg_diff,pageviews
0,user_1,64.400000,5
1,user_10,74.800000,5
2,user_11,2.000000,1
3,user_12,89.000000,4
4,user_13,48.500000,4
5,user_14,159.000000,3
6,user_15,17.500000,2
7,user_16,39.333333,3
8,user_17,61.600000,5
9,user_18,5.666667,3


Коэффициент корреляции - это статистический показатель, который используется для оценки степени связи между двумя переменными. С его помощью можно определить, насколько взаимосвязаны данные, исследуемые наблюдения или эксперименты. 

In [265]:
correlation = views_diff['avg_diff'].corr(views_diff['pageviews'])
print(correlation)

0.5319186090494608


In [ ]:
db.close()